# PsExec Tool Execution
Detects PsExec service installation and execution events (service and Sysmon)

## Rule Content
```
- action: global
  title: PsExec Tool Execution
  id: 42c575ea-e41e-41f1-b248-8093c3e82a28
  status: experimental
  description: Detects PsExec service installation and execution events (service and
    Sysmon)
  author: Thomas Patzke
  references:
  - https://www.jpcert.or.jp/english/pub/sr/ir_research.html
  - https://jpcertcc.github.io/ToolAnalysisResultSheet
  tags:
  - attack.execution
  - attack.t1035
  - attack.s0029
  detection:
    condition: 1 of them
  fields:
  - EventID
  - CommandLine
  - ParentCommandLine
  - ServiceName
  - ServiceFileName
  falsepositives:
  - unknown
  level: low
- logsource:
    product: windows
    service: system
  detection:
    service_installation:
      EventID: 7045
      ServiceName: PSEXESVC
      ServiceFileName: '*\PSEXESVC.exe'
    service_execution:
      EventID: 7036
      ServiceName: PSEXESVC
- logsource:
    category: process_creation
    product: windows
  detection:
    sysmon_processcreation:
      Image: '*\PSEXESVC.exe'
      User: NT AUTHORITY\SYSTEM

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(service_name:"PSEXESVC" AND ((event_id:"7045" AND service_image_path.keyword:*\\PSEXESVC.exe) OR event_id:"7036"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\PSEXESVC.exe AND user_account:"NT\ AUTHORITY\\SYSTEM")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()